In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# import CRUD Python module file
from animal_shelter import AnimalShelter


# CRUD Python module file name and class name
from animal_shelter import AnimalShelter




###########################
# Data Manipulation / Model
###########################
# username and password and CRUD Python module name
username = "aacuser"
password = "CS340"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    # HTML image tag
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 alt='customer image',
                 style ={
                     'height': '35%',
                     'width': '35%',
                 }))
    ], href = 'https://www.snhu.edu/'),
    #unique identifier
    html.Center(html.H5('Created by: Jessica Ayer, CS340 22EW1')),
    html.Hr(),
    #Code for the interactive filtering options.
    html.Div([
        dcc.RadioItems(id = 'radio_buttons',
             options = [{'label': 'All', 'value': 'all'},
                       {'label':'Water', 'value': 'water'},
                       {'label': 'Mountain/Wilderness', 'value': 'mountain'},
                       {'label': 'Disaster/Tracking', 'value': 'disaster'}
                       ],
                       style ={'color': 'DarkRed', 'font-size': 20},
             value = 'all')
              ]),
    html.Div(),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
 # the features for the interactive data table to make it user-friendly
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = 'single',
        column_selectable = False,
        row_selectable = False,
        row_deletable = True,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
        style_cell = {'minWidth': 150, 'maxWidth': 150, 'width': 150,
                     'textAlign': 'left'},
        fixed_rows = {'headers': True},
        style_header = {'overflow': 'hidden',
                       'textOverflow': 'ellipsis',
                        'maxWidth': 0,
                        'backgroundColor': 'rgb(30 ,30,30)',
                       'fontWeight': 'bold',
                       'color': 'white'
                       },
        style_as_list_view=True,  
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='map-id',
            className='col s12 m6',

            ),
        html.Div(
            id='graph-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id', 'columns')],
    [Input('radio_buttons', 'value')])
def update_dashboard(value):
    ### code to filter interactive data table with MongoDB queries
    if value == 'all':
        df = pd.DataFrame.from_records(shelter.read_all({}))
    elif value == 'water':
        df = pd.DataFrame.from_records(shelter.read_all(
        {'breed':{'$in':[
            'Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'
        ]},
        'sex_upon_outcome': 'Intact Female',
        'age_upon_outcome_in_weeks': {'$gte': 26}, '$and': [{'age_upon_outcome_in_weeks': {'$lte': 156}}]
        }))
    elif value == 'mountain':
        df = pd.DataFrame.from_records(shelter.read_all(
        {'breed':{'$in':[
            'German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'
        ]},
        'sex_upon_outcome': 'Intact Male',
        'age_upon_outcome_in_weeks': {'$gte': 26}, '$and': [{'age_upon_outcome_in_weeks': {'$lte': 156}}]
        }))
    elif value == 'disaster':
        df = pd.DataFrame.from_records(shelter.read_all(
        {'breed':{'$in':[
            'Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'
        ]},
        'sex_upon_outcome': 'Intact Male',
        'age_upon_outcome_in_weeks': {'$gte': 20}, '$and': [{'age_upon_outcome_in_weeks': {'$lte': 300}}]
        }))
    else:
        df = pd.DataFrame.from_records(shelter.read_all({}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # pie chart
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure = px.pie(dff, names = 'breed', hole= .3, color_discrete_sequence = px.colors.sequential.RdBu)
        )    
    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '500px', 'height': '400px'}, center=[30.75, -97.48], zoom=10, children = [
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75,-97.48], children=[
               dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1(dff.iloc[0,9]),
                    html.P(dff.iloc[0,3]),
                    html.P(dff.iloc[0,4]),
                    html.P(dff.iloc[0,1]),
                    html.P(dff.iloc[0,12])
                ])
            ])
        ])
    ]
app